# 테스트

In [18]:
import re
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

WHITESPACE_HANDLER = lambda k: re.sub('\s+', ' ', re.sub('\n+', ' ', k.strip()))

In [19]:
model_name = "csebuetnlp/mT5_m2m_crossSum_enhanced"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [20]:
from torchinfo import summary

summary(model)

Layer (type:depth-idx)                                       Param #
MT5ForConditionalGeneration                                  --
├─Embedding: 1-1                                             192,086,016
├─MT5Stack: 1-2                                              192,086,016
│    └─Embedding: 2-1                                        (recursive)
│    └─ModuleList: 2-2                                       --
│    │    └─MT5Block: 3-1                                    7,079,808
│    │    └─MT5Block: 3-2                                    7,079,424
│    │    └─MT5Block: 3-3                                    7,079,424
│    │    └─MT5Block: 3-4                                    7,079,424
│    │    └─MT5Block: 3-5                                    7,079,424
│    │    └─MT5Block: 3-6                                    7,079,424
│    │    └─MT5Block: 3-7                                    7,079,424
│    │    └─MT5Block: 3-8                                    7,079,424
│    │    └─MT5B

In [21]:
article_text = r'''
연차사용이 자유롭고 분위기가 느슨하다. 유연근무제 도입했다.
연봉은 최악이며 미래먹거리가 없으며 저마다 처신하기 바쁨.
제발 정신좀차리고 주인의식 좀 느끼세요. 그대들 배부르다고 직원들도 배부른줄아나?
자유로운 연차 사용으로 원할때 사용하면 되며, 직원들을 위해서 다양한 소통 문화를 만들려고 함.
개인 연차나 비활동시에도 시스템에서 바로 확인되었으면 좋겠고, 연차 시 확인이 안되서 업무 전화가 많이 옴.
회사 직원들의 의견을 더 들어주셨으면 좋겠고, 적극적으로 반영해주셨으면 좋겠습니다.
월급 안밀리고 밥나오고 연차를 눈치없이 쓸 수 있음.
오래다니면 다닐수록 일이 더 쉬워짐.
직원을 홀대하고 연봉을 올려주지 않는 반면 경력직들은 직전연봉 다 챙겨줘 가며 입사시킴.
바라는 것도 없고 그냔 이대로만 해주세요. 잘하고 있다면서요?
연차 자율제도와 유연근무 제도로 가정에 더 충실할 수 있고 삶이 생긴다.
한량? 들이 많고 일을 하려는 의지들이 약하며 다들 불만만 많은 상황인데 인사정책은 더 역효과를 불러옴.
본인들 평가나 성과를 위한 업무를 하지말고 밑에 사람들을 돌보아야 될듯.
'''

In [22]:
article_text = article_text.replace('\n', ' ').strip()

In [23]:
get_lang_id = lambda lang: tokenizer._convert_token_to_id(
    model.config.task_specific_params["langid_map"][lang][1]
) 

target_lang = "korean" # for a list of available language names see below

input_ids = tokenizer(
    [WHITESPACE_HANDLER(article_text)],
    return_tensors="pt",
    padding="max_length",
    truncation=True,
    max_length=512,
)["input_ids"]

Keyword arguments {'device': 'cuda'} not recognized.


In [29]:
model.to("cuda")
input_ids = input_ids.to("cuda")

In [30]:
output_ids = model.generate(
    input_ids=input_ids,
    decoder_start_token_id=get_lang_id(target_lang),
    max_length=84,
    no_repeat_ngram_size=1,
    num_beams=4,
)[0]

In [31]:
pred = tokenizer.decode(
    output_ids,
    skip_special_tokens=True,
    clean_up_tokenization_spaces=False
)

In [32]:
print(pred)

<extra_id_70> 회사에서 연차 사용이 자유롭고 분위기가 느슨하다.


In [33]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn

In [ ]:
def get_sae4k_df(path='/home/parking/ml/data/MiniProj/data/sae4k/sae4k_v2.txt'):
    

In [ ]:
class Sae4KDataset(Dataset):
    def __init__(self, tokenizer, df):
        self.__tokenizer = tokenizer
        self.__df = df

# 모듈화 된 모델 테스트

In [10]:
from modules.torchmodules.mt5sum import MT5Sum

mt5sum = MT5Sum()

In [2]:
article_text = r'''
SK하이닉스 제공SK하이닉스는 SK스퀘어가 효율적인 해외 반도체 투자를 위해 설립한 투자법인 'TGC SQUARE'에 공동 출자했다고 4일 밝혔다. 투자법인은 1천억 원을 시작으로 추가 참여를 원하는 기업을 위해 공정출자 기회를 열어 두고 있다. SK하이닉스를 비롯해 신한금융그룹과 LIG넥스원 등이 공동 출자에 참여했다. SK하이닉스와 SK스퀘어는 독보적인 기술력을 가진 반도체 소부장 기업에 선제적으로 투자해 안정적인 글로벌 반도체 공급망을 구축하고, 첨단 기술 경쟁력을 강화할 계획이다. 글로벌 반도체 공급망은 경쟁적으로 자국 중심의 생태계 조성을 위해 급격하게 재편하고 있다. 이에 따라 글로벌 밸류체인 강화를 위해 △반도체 설계 △생산 △패키징 공정별 기술적 우위를 가진 소부장 기업화 협력이 필수라는 평가다. TGC SQUARE는 글로벌 탑티어 반도체 기업의 전문가가 기술 인사이트를 제공하는 '반도체 자문위원회'를 운영해 전문적인 투자심의 체계를 구축했다. 최우성 SK스퀘어 반도체 투자담당(MD) 겸 SK텔레콤 재팬 대표가 투자법인의 CEO(최고경영자)를 맡는다. 조희준 전 BNP파리바 일본법인 영업담당을 CIO(최고투자책임자)로, 미야모토 야스테루 전 크레디트스위스 부사장을 전문심사역으로 영입했다. SK하이닉스와 SK스퀘어는 첫 투자 대상으로 일본 반도체 강소기업을 검토하고 있다. 현재 조성된 투자금의 60%를 일본 소부장 기업에 투자한다는 방침이다. 일본은 전 세계 반도체 소부장 강자로 꼽힌다. △반도체 소재 △부품 △장비 등 소부장 영역에서 대체가 어려운 하이엔드 기술에 특화해 전 세계시장점유율 30%대를 차지하는 글로벌 1~2위 기업이 많다. SK하이닉스와 SK스퀘어는 일본 반도체 투자 네트워크를 가동해 △반도체 검사장비 개발 A사 △친환경 반도체 부품 제조 B사 △AI(인공지능) 반도체 개발 C사 △차세대 반도체 소재 개발 D사 등 잠재적 투자 대상 기업을 중심으로 기술검증을 진행할 예정이다. SK하이닉스와 SK스퀘어는 성장기업에 투자한 이후 향후 M&A(인수합병)와 IPO(기업공개)를 지원하는 방식도 검토하고 있다. TGC SQUARE 최우성 CEO는 "글로벌 반도체 인사이트를 가진 SK 주요 관계사와 국내 대표 금융사 등이 해외 공동투자를 통해 국내외 반도체 산업의 생태계를 확장하는 유의미한 프로젝트"라며 "글로벌 유수의 소부장 기업과 협력을 확대해 미래 반도체 기술 기반을 다지는 계기가 될 것"이라고 말했다.
'''

In [5]:
mt5sum.get_sum(article_text)

'<extra_id_70> 글로벌 반도체 소부장 기업 SK하이닉스가 세계 최대 규모의 해외 공정출자 기회를 열었다.'

In [9]:
import pandas as pd

news_df = pd.read_csv('/home/parking/ml/data/MiniProj/data/news/SK 하이닉스_naver.csv')

In [11]:
news_df['sum'] = news_df['content'].apply(lambda x: mt5sum.get_sum(x))
news_df['sum'] = news_df['sum'].apply(lambda x: x.replace('<extra_id_70>', '').strip())
news_df.head()

/home/parking/anaconda3/envs/ezpz/lib/python3.10/site-packages/torch/cuda/__init__.py:546: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


,link,content,sum
0,https://www.nocutnews.co.kr/news/5970546,SK하이닉스 제공SK하이닉스는 SK스퀘어가 효율적인 해외 반도체 투자를 위해 설립한...,글로벌 반도체 소부장 기업 SK하이닉스가 세계 최대 규모의 해외 공정출자 기회를 열었다.
1,https://www.etnews.com/20230704000192,SK가 일본 반도체 소재·부품·장비(소부장) 기술 확보에 나선다. SK하이닉스와 S...,글로벌 반도체 기업 SK하이닉스(SK)가 일본 소부장 기술 확보에 나선다.
2,https://www.hankyung.com/finance/article/20230...,"사진=한경DB 이베스트투자증권은 5일 SK하이닉스 SK하이닉스 117,300 -0....",지난 5일 이베스트 투자증권은 SK하이닉스(S&P)의 실적 추정치는 매출 7조300...
3,http://news.mt.co.kr/mtview.php?no=20230704103...,SK하이닉스와 SK스퀘어가 국내 대표 금융사 등과 약 1000억원을 공동 출자해 일...,글로벌 반도체 기업 SK하이닉스가 일본과 미국 등 해외 유망기업에 약 1000억원을...
4,https://n.news.naver.com/mnews/article/092/000...,투자법인 TGC SQUARE 설립...첫 투자 대상은 일본 소부장 기업SK하이닉스와...,일본의 반도체 소부장 기업 SK하이닉스(SK텔레콤 재팬)가 국내 대표 금융사 등과 ...


In [13]:
news_df.to_csv('/home/parking/ml/data/MiniProj/data/news/SK 하이닉스_naver_sum.csv', index=False, encoding='utf-8-sig')

# bert모델

In [59]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch.nn.functional as F

In [60]:
tokenizer = AutoTokenizer.from_pretrained("snunlp/KR-FinBert-SC")
model = AutoModelForSequenceClassification.from_pretrained("snunlp/KR-FinBert-SC")

In [70]:
input_str = 'SK하이닉스가 해외 유망 소부장 기업을 최소해 투자할 계획이다.'

In [71]:
input = tokenizer(input_str, return_tensors='pt')
output = model(**input)

In [72]:
sm_output = F.softmax(output.logits, dim=1)

In [76]:
sm_output.argmax(dim=1)

tensor([2])

In [77]:
model.config.id2label[sm_output.argmax(dim=1).item()]

'positive'

In [79]:
from modules.torchmodules.FinBertKR import FinBertKR

fbkr = FinBertKR()

In [80]:
fbkr.predict(input_str)

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper_CUDA__index_select)

In [15]:
df = pd.read_csv('/home/parking/ml/data/MiniProj/data/news/SK 하이닉스_naver_sum.csv', encoding='utf-8-sig')
df.head()

,link,content,sum
0,https://www.nocutnews.co.kr/news/5970546,SK하이닉스 제공SK하이닉스는 SK스퀘어가 효율적인 해외 반도체 투자를 위해 설립한...,글로벌 반도체 소부장 기업 SK하이닉스가 세계 최대 규모의 해외 공정출자 기회를 열었다.
1,https://www.etnews.com/20230704000192,SK가 일본 반도체 소재·부품·장비(소부장) 기술 확보에 나선다. SK하이닉스와 S...,글로벌 반도체 기업 SK하이닉스(SK)가 일본 소부장 기술 확보에 나선다.
2,https://www.hankyung.com/finance/article/20230...,"사진=한경DB 이베스트투자증권은 5일 SK하이닉스 SK하이닉스 117,300 -0....",지난 5일 이베스트 투자증권은 SK하이닉스(S&P)의 실적 추정치는 매출 7조300...
3,http://news.mt.co.kr/mtview.php?no=20230704103...,SK하이닉스와 SK스퀘어가 국내 대표 금융사 등과 약 1000억원을 공동 출자해 일...,글로벌 반도체 기업 SK하이닉스가 일본과 미국 등 해외 유망기업에 약 1000억원을...
4,https://n.news.naver.com/mnews/article/092/000...,투자법인 TGC SQUARE 설립...첫 투자 대상은 일본 소부장 기업SK하이닉스와...,일본의 반도체 소부장 기업 SK하이닉스(SK텔레콤 재팬)가 국내 대표 금융사 등과 ...


In [16]:
df['sentiment'] = df['sum'].apply(lambda x: classifier(x)[0]['label'])
df.head()

,link,content,sum,sentiment
0,https://www.nocutnews.co.kr/news/5970546,SK하이닉스 제공SK하이닉스는 SK스퀘어가 효율적인 해외 반도체 투자를 위해 설립한...,글로벌 반도체 소부장 기업 SK하이닉스가 세계 최대 규모의 해외 공정출자 기회를 열었다.,negative
1,https://www.etnews.com/20230704000192,SK가 일본 반도체 소재·부품·장비(소부장) 기술 확보에 나선다. SK하이닉스와 S...,글로벌 반도체 기업 SK하이닉스(SK)가 일본 소부장 기술 확보에 나선다.,negative
2,https://www.hankyung.com/finance/article/20230...,"사진=한경DB 이베스트투자증권은 5일 SK하이닉스 SK하이닉스 117,300 -0....",지난 5일 이베스트 투자증권은 SK하이닉스(S&P)의 실적 추정치는 매출 7조300...,negative
3,http://news.mt.co.kr/mtview.php?no=20230704103...,SK하이닉스와 SK스퀘어가 국내 대표 금융사 등과 약 1000억원을 공동 출자해 일...,글로벌 반도체 기업 SK하이닉스가 일본과 미국 등 해외 유망기업에 약 1000억원을...,negative
4,https://n.news.naver.com/mnews/article/092/000...,투자법인 TGC SQUARE 설립...첫 투자 대상은 일본 소부장 기업SK하이닉스와...,일본의 반도체 소부장 기업 SK하이닉스(SK텔레콤 재팬)가 국내 대표 금융사 등과 ...,negative


In [17]:
df.to_csv('/home/parking/ml/data/MiniProj/data/news/SK 하이닉스_naver_sum_sentiment.csv', index=False, encoding='utf-8-sig')